# Import Modules


In [1]:
import pandas as pd
import numpy as np
import sys
import os
import warnings
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, normalize
from sklearn import metrics
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
import seaborn as sns

sys.path.append(os.path.abspath(os.path.join('../scripts')))

from clean_telecom_dataframe import Telecom
from plot_telecom_dataframe import TelecomPlotter

plot = TelecomPlotter()
cleaner = Telecom()


# Read CSV Data


In [2]:
telecom_clean_df = pd.read_csv("../data/telecom_cleaned_data.csv")

In [4]:
# Relevant columns for exprience analysis
telecom_exprience_df = telecom_clean_df[['MSISDN/Number','Avg RTT DL (ms)','Avg RTT UL (ms)','Avg Bearer TP DL (kbps)','Avg Bearer TP UL (kbps)','TCP DL Retrans. Vol (Bytes)','TCP UL Retrans. Vol (Bytes)','Handset Type']]
telecom_exprience_df

,MSISDN/Number,Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps),TCP DL Retrans. Vol (Bytes),TCP UL Retrans. Vol (Bytes),Handset Type
0,33664962239,42.000000,5.000000,23.0,44.0,2.080991e+07,759658.664811,Samsung Galaxy A5 Sm-A520F
1,33681854413,65.000000,5.000000,16.0,26.0,2.080991e+07,759658.664811,Samsung Galaxy J5 (Sm-J530)
2,33760627129,109.795706,17.662883,6.0,9.0,2.080991e+07,759658.664811,Samsung Galaxy A8 (2018)
3,33699795932,109.795706,17.662883,6.0,9.0,2.080991e+07,759658.664811,Samsung Sm-G390F
4,33665368271,102.000000,5.000000,22.0,38.0,2.080991e+07,759658.664811,Huawei Mate 20 Lite
...,...,...,...,...,...,...,...,...
138066,33645655643,109.795706,17.662883,55.0,54.0,2.080991e+07,759658.664811,Samsung Galaxy J5 (Sm-J530)
138067,33650688697,32.000000,0.000000,52.0,65.0,2.080991e+07,759658.664811,Apple iPhone 8 Plus (A1897)
138068,33663449963,27.000000,2.000000,23.0,54.0,2.080991e+07,759658.664811,Apple iPhone Se (A1723)
138069,33621890103,43.000000,6.000000,43.0,47.0,2.080991e+07,759658.664811,Apple iPhone Xs (A2097)


In [5]:
# merge Avg RTT (ms) , Avg Bearer TP (kbps) and Total TCP Retrans. Vol (Bytes)
telecom_exprience_df['Total Avg RTT (ms)'] = telecom_exprience_df['Avg RTT DL (ms)'] + telecom_exprience_df['Avg RTT UL (ms)']
telecom_exprience_df['Total Avg Bearer TP (kbps)'] = telecom_exprience_df['Avg Bearer TP DL (kbps)'] + telecom_exprience_df['Avg Bearer TP UL (kbps)']
telecom_exprience_df['Total TCP Retrans. Vol (Bytes)'] = telecom_exprience_df['TCP DL Retrans. Vol (Bytes)'] + telecom_exprience_df['TCP UL Retrans. Vol (Bytes)']
telecom_exprience_df.head()

/tmp/ipykernel_32657/1313543884.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  telecom_exprience_df['Total Avg RTT (ms)'] = telecom_exprience_df['Avg RTT DL (ms)'] + telecom_exprience_df['Avg RTT UL (ms)']
/tmp/ipykernel_32657/1313543884.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  telecom_exprience_df['Total Avg Bearer TP (kbps)'] = telecom_exprience_df['Avg Bearer TP DL (kbps)'] + telecom_exprience_df['Avg Bearer TP UL (kbps)']
/tmp/ipykernel_32657/1313543884.py:4: SettingWithCopyWarning: 
A v

,MSISDN/Number,Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps),TCP DL Retrans. Vol (Bytes),TCP UL Retrans. Vol (Bytes),Handset Type,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes)
0,33664962239,42.000000,5.000000,23.0,44.0,2.080991e+07,759658.664811,Samsung Galaxy A5 Sm-A520F,47.000000,67.0,2.156957e+07
1,33681854413,65.000000,5.000000,16.0,26.0,2.080991e+07,759658.664811,Samsung Galaxy J5 (Sm-J530),70.000000,42.0,2.156957e+07
2,33760627129,109.795706,17.662883,6.0,9.0,2.080991e+07,759658.664811,Samsung Galaxy A8 (2018),127.458589,15.0,2.156957e+07
3,33699795932,109.795706,17.662883,6.0,9.0,2.080991e+07,759658.664811,Samsung Sm-G390F,127.458589,15.0,2.156957e+07
4,33665368271,102.000000,5.000000,22.0,38.0,2.080991e+07,759658.664811,Huawei Mate 20 Lite,107.000000,60.0,2.156957e+07


In [6]:
telecom_exprience_df = telecom_exprience_df[['MSISDN/Number','Total Avg RTT (ms)','Total Avg Bearer TP (kbps)','Total TCP Retrans. Vol (Bytes)','Handset Type']]
telecom_exprience_df.head()

,MSISDN/Number,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes),Handset Type
0,33664962239,47.000000,67.0,2.156957e+07,Samsung Galaxy A5 Sm-A520F
1,33681854413,70.000000,42.0,2.156957e+07,Samsung Galaxy J5 (Sm-J530)
2,33760627129,127.458589,15.0,2.156957e+07,Samsung Galaxy A8 (2018)
3,33699795932,127.458589,15.0,2.156957e+07,Samsung Sm-G390F
4,33665368271,107.000000,60.0,2.156957e+07,Huawei Mate 20 Lite


In [7]:
tellco_exprience_df1 = telecom_exprience_df.groupby(
    'MSISDN/Number').agg({'Total Avg RTT (ms)': 'sum', 'Total Avg Bearer TP (kbps)': 'sum', 'Total TCP Retrans. Vol (Bytes)': 'sum','Handset Type': [lambda x: x.mode()[0]] }) #' '.join(x)
tellco_exprience_df1

,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes),Handset Type
,sum,sum,sum,<lambda>
MSISDN/Number,,,,
33601001722,46.000000,76.0,2.156957e+07,Huawei P20 Lite Huawei Nova 3E
33601001754,31.000000,99.0,2.156957e+07,Apple iPhone 7 (A1778)
33601007832,84.000000,248.0,7.607247e+05,Apple iPhone 5S (A1457)
33601008617,119.000000,56844.0,3.094040e+07,Apple iPhone Se (A1723)
33601010682,151.458589,7908.0,2.233199e+07,Samsung Galaxy A8 (2018)
...,...,...,...,...
33789960306,127.458589,106.0,2.156957e+07,Samsung Galaxy S8 Plus (Sm-G955F)
33789967113,53.000000,67.0,2.156957e+07,Apple iPhone 6S (A1688)


In [8]:
telecom_exprience_df = pd.DataFrame(columns=[
    "Total Avg RTT (ms)",
    "Total Avg Bearer TP (kbps)",
    "Total TCP Retrans. Vol (Bytes)",
    "Handset Type"])

telecom_exprience_df["Total Avg RTT (ms)"] = tellco_exprience_df1["Total Avg RTT (ms)"]['sum']
telecom_exprience_df["Total Avg Bearer TP (kbps)"] = tellco_exprience_df1["Total Avg Bearer TP (kbps)"]['sum']
telecom_exprience_df["Total TCP Retrans. Vol (Bytes)"] = tellco_exprience_df1["Total TCP Retrans. Vol (Bytes)"]['sum']
telecom_exprience_df["Handset Type"] = tellco_exprience_df1["Handset Type"]['<lambda>']
telecom_exprience_df.head()

,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes),Handset Type
MSISDN/Number,,,,
33601001722,46.000000,76.0,2.156957e+07,Huawei P20 Lite Huawei Nova 3E
33601001754,31.000000,99.0,2.156957e+07,Apple iPhone 7 (A1778)
33601007832,84.000000,248.0,7.607247e+05,Apple iPhone 5S (A1457)
33601008617,119.000000,56844.0,3.094040e+07,Apple iPhone Se (A1723)
33601010682,151.458589,7908.0,2.233199e+07,Samsung Galaxy A8 (2018)


In [9]:
#  Top, bottom and most frequent Datas Top 10


In [15]:
tar = telecom_exprience_df.sort_values('Total Avg RTT (ms)', ascending=False)
tar.head(10)['Total Avg RTT (ms)']

MSISDN/Number
33662317023    96924.0
33660874265    64670.0
33683692867    54848.0
33698551167    46021.0
33761813523    37084.0
33668791629    36304.0
33760941100    27278.0
33606788933    26300.0
33671816754    25715.0
33781865588    25388.0
Name: Total Avg RTT (ms), dtype: float64

In [11]:
telecom_exprience_df['Total Avg RTT (ms)'].value_counts().head(10)


127.458589    13281
29.000000      2607
39.000000      2054
254.917178     1588
38.000000      1370
30.000000      1364
40.000000      1315
28.000000      1275
49.000000      1131
31.000000      1015
Name: Total Avg RTT (ms), dtype: int64

In [16]:
tab = telecom_exprience_df.sort_values('Total Avg Bearer TP (kbps)', ascending=False)
tab

,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes),Handset Type
MSISDN/Number,,,,
33762333464,400.000000,902222.0,2.788275e+07,Samsung Galaxy A8 (2018)
33668708263,500.000000,840424.0,1.429871e+08,Huawei B528S-23A
33659546392,588.000000,798342.0,3.472787e+07,Huawei B528S-23A
33669946573,370.000000,690815.0,6.271756e+07,Huawei B528S-23A
33668425947,382.000000,676858.0,7.876596e+08,Huawei B528S-23A
...,...,...,...,...
33659471143,127.458589,0.0,2.156957e+07,Huawei P20 Pro
33667414355,127.458589,0.0,2.156957e+07,Samsung Galaxy S (Gt-I9000)
33761309821,141.795706,0.0,2.156957e+07,Lg Lg-T385


In [17]:
tab.head(10)['Total Avg Bearer TP (kbps)']

MSISDN/Number
33762333464    902222.0
33668708263    840424.0
33659546392    798342.0
33669946573    690815.0
33668425947    676858.0
33625700673    564179.0
33658075438    550352.0
33762644658    549661.0
33665658364    530353.0
33666461685    524686.0
Name: Total Avg Bearer TP (kbps), dtype: float64

In [18]:
tcp_retransmitted = telecom_exprience_df.sort_values('Total TCP Retrans. Vol (Bytes)', ascending=False)
tcp_retransmitted

,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes),Handset Type
MSISDN/Number,,,,
33667725464,703.0,379713.0,8.354209e+09,Huawei B528S-23A
33659084281,396.0,210259.0,7.941860e+09,Huawei B528S-23A
33660977522,417.0,216852.0,7.374899e+09,Huawei B528S-23A
33761284050,351.0,144570.0,6.043978e+09,Huawei B528S-23A
33762644658,2860.0,549661.0,5.879989e+09,Huawei B528S-23A
...,...,...,...,...
33607833304,32.0,16853.0,1.430000e+02,Apple iPhone 5C (A1529)
33626357513,40.0,10964.0,1.340000e+02,Apple iPhone 7 (A1778)
33762062356,41.0,32577.0,1.290000e+02,Apple iPhone 8 (A1905)


In [19]:
tcp_retransmitted.head(10)['Total TCP Retrans. Vol (Bytes)']

MSISDN/Number
33667725464    8.354209e+09
33659084281    7.941860e+09
33660977522    7.374899e+09
33761284050    6.043978e+09
33762644658    5.879989e+09
33650658056    5.593366e+09
33663180885    5.566929e+09
33669054076    5.195785e+09
33763572381    5.179459e+09
33658638625    5.165857e+09
Name: Total TCP Retrans. Vol (Bytes), dtype: float64

In [20]:
tcp_retransmitted.tail(10)['Total TCP Retrans. Vol (Bytes)']

MSISDN/Number
33699694715    182.0
33679366089    179.0
33784203839    177.0
33769636458    176.0
33650681239    176.0
33607833304    143.0
33626357513    134.0
33762062356    129.0
33614777138    128.0
33650128412     97.0
Name: Total TCP Retrans. Vol (Bytes), dtype: float64

In [21]:
telecom_exprience_df['Total TCP Retrans. Vol (Bytes)'].value_counts().head(10)

2.156957e+07    42973
4.313915e+07     6850
6.470872e+07      948
2.081121e+07      356
8.627829e+07      229
2.081124e+07      150
7.609887e+05      141
4.238078e+07       81
2.081123e+07       78
7.597507e+05       76
Name: Total TCP Retrans. Vol (Bytes), dtype: int64

In [22]:
telecom_exprience_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99115 entries, 33601001722 to 337000037000919
Data columns (total 4 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Total Avg RTT (ms)              99115 non-null  float64
 1   Total Avg Bearer TP (kbps)      99115 non-null  float64
 2   Total TCP Retrans. Vol (Bytes)  99115 non-null  float64
 3   Handset Type                    99115 non-null  object 
dtypes: float64(3), object(1)
memory usage: 3.8+ MB


# Handset Analysis


In [23]:
handset_type_exprience_df = telecom_exprience_df.groupby('Handset Type').agg({'Total Avg Bearer TP (kbps)': 'mean', 'Total TCP Retrans. Vol (Bytes)': 'mean','Total Avg RTT (ms)': "mean"}) #.mean()
handset_type_exprience_df #.sort_values(by='Total Avg Bearer TP (kbps)', ascending=False).head()

,Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes),Total Avg RTT (ms)
Handset Type,,,
A-Link Telecom I. Cubot A5,23510.0,2.156957e+07,42.000000
A-Link Telecom I. Cubot Note Plus,6699.0,6.162130e+05,1589.000000
A-Link Telecom I. Cubot Note S,8937.0,4.141173e+07,890.000000
A-Link Telecom I. Cubot Nova,56217.0,1.371700e+05,44.000000
A-Link Telecom I. Cubot Power,69468.0,8.041000e+03,43.000000
...,...,...,...
Zte Racer Iii Mini Zte Switch X1 Bouygues Telecom Bs 402 Blade Q Mini Zte Blade G Pro,1549.0,8.789197e+06,227.400000
Zte Zte Blade C2 Smartphone Android By Sfr Startrail 4 Zte Blade Flex T809 Zte T809,58.0,1.116595e+07,165.729294
Zyxel Communicat. Lte7460,61956.0,4.127655e+07,133.000000


In [24]:

#average TCP retransmission view per handset type
handset_type_exprience_df.sort_values('Total TCP Retrans. Vol (Bytes)', ascending=False).head()

,Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes),Total Avg RTT (ms)
Handset Type,,,
Lg Lg-H635,25339.0,9.125491e+08,80.333333
Huawei Bln-Al10,5761.0,3.291746e+08,292.000000
Asustek Asus Zenfone Selfie Zd551Kl,27585.0,3.216583e+08,121.500000
Samsung Galaxy Tab S3 (Sm-T825),34720.0,2.534953e+08,121.500000
Huawei E5776S-32,31572.5,2.110367e+08,169.750000


# K-Means Clustering

In [25]:
telecom_exprience_df.drop(['Handset Type'], axis=1, inplace=True)


In [26]:
scale_data = StandardScaler().fit_transform(telecom_exprience_df)
scale_data

array([[-0.18757153, -0.49997097, -0.05714637],
       [-0.20961963, -0.49943776, -0.05714637],
       [-0.13171635, -0.49598343, -0.18790118],
       ...,
       [-0.17875229, -0.26143688, -0.19131117],
       [-0.19639077, -0.49904364, -0.05714637],
       [-0.06783773, -0.50096786, -0.05714637]])

In [28]:
normalized_data = normalize(scale_data)
normalized_data

array([[-0.34926448, -0.93096271, -0.10640846],
       [-0.38486999, -0.9169876 , -0.10492302],
       [-0.2410207 , -0.90757354, -0.34383031],
       ...,
       [-0.48311151, -0.70658207, -0.51705421],
       [-0.3641367 , -0.92529858, -0.10595758],
       [-0.13333943, -0.98468458, -0.11232488]])

In [29]:
kmeans = KMeans(n_clusters = 3, random_state = 42).fit(normalized_data)
y_kmeans = kmeans.fit_predict(normalized_data)
X = np.array(normalized_data)
y_kmeans

array([0, 0, 0, ..., 0, 0, 0], dtype=int32)

In [32]:
clustered_telecom_exprience_df = telecom_exprience_df.copy()
clustered_telecom_exprience_df.insert(0, 'Cluster', y_kmeans)
clustered_telecom_exprience_df

,Cluster,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes)
MSISDN/Number,,,,
33601001722,0,46.000000,76.0,2.156957e+07
33601001754,0,31.000000,99.0,2.156957e+07
33601007832,0,84.000000,248.0,7.607247e+05
33601008617,1,119.000000,56844.0,3.094040e+07
33601010682,0,151.458589,7908.0,2.233199e+07
...,...,...,...,...
33789960306,0,127.458589,106.0,2.156957e+07
33789967113,0,53.000000,67.0,2.156957e+07
33789996170,0,52.000000,10365.0,2.180450e+05


# User segmentation into group of experiences


In [33]:
clustered_telecom_exprience_df[clustered_telecom_exprience_df["Cluster"]==0][["Total Avg RTT (ms)","Total Avg Bearer TP (kbps)","Total TCP Retrans. Vol (Bytes)"]].describe()

,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes)
count,65298.000000,65298.000000,6.529800e+04
mean,86.279836,2052.331312,2.203945e+07
std,58.931085,4675.518350,1.398449e+07
min,0.000000,0.000000,1.280000e+02
25%,39.000000,88.000000,2.156957e+07
50%,66.000000,113.000000,2.156957e+07
75%,127.458589,484.000000,2.156957e+07
max,318.000000,21927.000000,3.453570e+08


In [34]:
clustered_telecom_exprience_df[clustered_telecom_exprience_df["Cluster"]==2][["Total Avg RTT (ms)","Total Avg Bearer TP (kbps)","Total TCP Retrans. Vol (Bytes)"]].describe()


,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes)
count,6725.000000,6725.000000,6.725000e+03
mean,1034.029856,11223.352119,5.724503e+07
std,2414.449035,14762.677060,3.404327e+08
min,46.000000,1.000000,1.403000e+03
25%,334.000000,567.000000,7.631377e+05
50%,472.000000,6430.000000,5.873960e+06
75%,974.000000,17689.000000,2.278511e+07
max,96924.000000,209835.000000,6.043978e+09


In [35]:
clustered_telecom_exprience_df.to_csv('../data/telecom_user_experience_data.csv')